In [ ]:
import numpy as np
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
from data_loaders import *
from missing_process.block_rules import *
import json
import matplotlib.pyplot as plt
import math
from tqdm import tqdm

import numpy as np
from scipy.stats import pearsonr,kendalltau

In [ ]:


real_datalist = [
    "banknoe",
        "contcrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]

missingtypelist = [
                    "quantile",
                   "diffuse",
                   "logistic"
                   "mcar","mar"
                   ]

seed = 1
nfold = 5

In [ ]:
def load_impute_data(missingtype,model_name,rule_name,dataname,fold,seed = 1):

    train_impute = np.load(f'impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_train.npy')
    test_impute = np.load(f'impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_test.npy')
    return train_impute,test_impute

def load_train_test(index_file,norm_values,observed_masks):  

    train_index = index_file["train_index"]
    test_index = index_file["test_index"]

    train_values = norm_values[train_index,:]

    train_masks = observed_masks[train_index,:]

    test_values = norm_values[test_index,:]

    test_masks = observed_masks[test_index,:]


    return train_values,train_masks,test_values,test_masks

def fillin_imputed_data(imputed,mask,original):
    filled_data = np.where(mask == 1, original, imputed)
    return filled_data


def find_missing_column(mask):
    columns_with_all_missing_values = []
    #for column in mask.columns:
    for column in range(mask.shape[1]):
        if mask[:,column].any():
            columns_with_all_missing_values.append(column)
    return columns_with_all_missing_values


In [ ]:

datalist = real_datalist


def run_correlation(rule_name,model_name_list,missingtype,datalist):

    d_corr = {}

    for dataname in tqdm(datalist):
        
        directory_path = f"datasets/{dataname}"
        data = dataset_loader(dataname)    
        norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
        
        observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')
        f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
        index_file = json.load(f)

        d_corr[dataname] = {}


        for model_name in model_name_list:
            d_corr[dataname][model_name]=0


            corr_final_list = []
            for fold in index_file:
                index = index_file[fold]
                train_values,train_masks,test_values,test_masks = load_train_test(index,norm_values,observed_masks)
                impute_train,impute_test  = load_impute_data(missingtype,model_name,rule_name,dataname,fold)

                impute_test = fillin_imputed_data(impute_test,test_masks,test_values)
                impute_test = np.nan_to_num(impute_test, nan=0)

                test_masks = 1-test_masks

                columns = find_missing_column(test_masks)
                
                complete = test_masks * test_values
                impute = test_masks * impute_test

                corr_list = []



                for index in columns:

                    complete_array = complete[:,index]
                    impute_array = impute[:,index]
                    corr,_ = pearsonr(complete_array,impute_array)
                    #corr,_ = kendalltau(complete_array,impute_array)
                    if math.isnan(corr) or model_name == "mean":
                        corr = 0
                    corr_list.append(corr)



                corr_final_list.append(np.mean(corr_list))

            

            d_corr[dataname][model_name]=np.mean(corr_final_list)



    d_corr = pd.DataFrame(d_corr).T
    new_col_names = ["RD", "ZR", "Mean", "KNN", "MF", "Mice", "MisF", "XGB", "OT", "HI", "GAIN", "Mi", "NMi", "CSDI"]
    new_row_names = ["Bank","Cali","Climate","Concre","Qsar",  "Red",  "Sonar", "White", "Yacht","Yeast"]

    d_corr = d_corr.rename(index=dict(zip(d_corr.index, new_row_names)), columns=dict(zip(d_corr.columns, new_col_names)))




In [ ]:
model_name_list = ["random", "zero", "mean", "knn", "mf", "mice", "missforest", "XGB", "ot", "hyper", "gain", "miwae", "notmiwae", "tabcsdi"]


real_datalist = ["banknote",
            "california","climate_model_crashes","concrete_compression",
           "qsar_biodegradation","wine_quality_red", "connectionist_bench_sonar","wine_quality_white",
            "yacht_hydrodynamics","yeast"
            ]

for missingtype in ["diffuse","logistic"]:
    for rule_name in ["0.3","0.5","0.7"]:
        run_correlation(rule_name,model_name_list,missingtype,real_datalist)


for missingtype in ["mar","mcar"]:
    for rule_name in ["0.3","0.5","0.7"]:
        run_correlation(rule_name,model_name_list,missingtype,real_datalist)


missingtype = "quantile"
for rule_name in ["Q1_Q4_0.5","Q2_Q3_0.5","Q2_Q4_0.5"]:
    run_correlation(rule_name,model_name_list,missingtype,real_datalist)